In [ ]:
from utils import *
import os
import shutil
# convert_annotation
suffix = ['.jpg', '.png', '.PNG', '.JPG', '.jpeg', '.JPEG']
for l in os.listdir('label')[1:-1]:
    nn=l[0:-4]
    for s in suffix:
        if os.path.exists('data/image/'+nn+s):
            shutil.move('data/image/'+nn+s,'data/train_image/'+nn+'.png')
    convert_annotation(nn)
    print(nn)

In [7]:
for l in os.listdir('image'):
    s = l[-4:]
    n = l[0:-4]
    if s in suffix:
        if os.path.exists('data/train_image/'+n+'.png'):
            os.remove('data/image/'+l)
        else:
            os.rename('data/image/'+l, 'data/image/'+n+'.png')

In [22]:
import random
import os
trainval_percent = 0.95  # 训练和验证集所占比例，剩下的0.1就是测试集的比例
train_percent = 0.9  # 训练集所占比例，可自己进行调整
total_xml = os.listdir('train_label')

num = len(total_xml)
listt = range(num)
tv = int(num * trainval_percent)
tr = int(tv * train_percent)
trainval = random.sample(listt, tv)
train = random.sample(trainval, tr)

ftrainval = open('../coco/trainval.txt', 'w')
ftest = open('../coco/test.txt', 'w')
ftrain = open('../coco/train.txt', 'w')
fval = open('../coco/val.txt', 'w')

for i in listt:
    name = 'E:/Jupyter/coco/images/'+total_xml[i][:-4]+'.jpg' + '\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)

ftrainval.close()
ftrain.close()
fval.close()
ftest.close()


In [42]:
import pandas as pd
dataset = pd.read_excel('data/2021MCMProblemC_DataSet.xlsx')
dataset.head(1)

,GlobalID,Detection Date,Notes,Lab Status,Lab Comments,Submission Date,Latitude,Longitude
0,{5AC8034E-5B46-4294-85F0-5B13117EBEFE},2019-12-08 00:00:00,"One dead wasp seen in Blaine, and suspect flyi...",Positive ID,,2020-01-15,48.980994,-122.688503


In [43]:
imglist = pd.read_excel('data/2021MCM_ProblemC_ Images_by_GlobalID.xlsx', usecols=['FileName', 'GlobalID'])
imglist.head(1)

,FileName,GlobalID
0,ATT1_DSCN9647.jpg,{5AC8034E-5B46-4294-85F0-5B13117EBEFE}


In [44]:
img_sample_raw = imglist[['FileName', 'GlobalID']].merge(dataset[['GlobalID', 'Lab Status']], on='GlobalID')
img_sample_raw['FileName'] = img_sample_raw['FileName'].apply(lambda x: x[0:x.rfind('.')] if x.rfind('.')!=-1 else)
img_sample_raw.set_index('FileName', drop=False, inplace=True, verify_integrity=True)
img_sample_raw.head(1)

,FileName,GlobalID,Lab Status
FileName,,,
ATT1_DSCN9647,ATT1_DSCN9647,{5AC8034E-5B46-4294-85F0-5B13117EBEFE},Positive ID


In [48]:
import os
import cv2
import re
il = os.listdir('image')
cnt = 0
sub_img = pd.DataFrame(columns=['id', 'class', 'FileName', 'GlobalID'])
for filename in il:
    suffix_pos = filename.rfind('.')
    placeholder = filename.rfind('+')
    if placeholder == -1 or filename[0:suffix_pos] in img_sample_raw.index:
        s = filename[0:suffix_pos]
    else:
        s = filename[0:placeholder]
    GlobalID = None
    # Get Class
    if s.startswith('positive'):
        label = 1
    elif s.startswith('negative'):
        label = 0
    else:
        GlobalID = img_sample_raw.loc[s, 'GlobalID']
        raw = img_sample_raw.loc[s, 'Lab Status']
        label = 1 if raw == 'Positive ID' else (0 if raw == 'Negative ID' else -1)
    # Check if image exists
    if not os.path.exists('data/label/'+s+'.txt'):
        continue
    src = cv2.imread('data/image/'+filename)
    # load image
    h, w = src.shape[0:2]
    with open('data/label/'+s+'.txt', 'r') as box_file:
        for line in box_file.readlines():
            # Get box position
            line = line.strip('\n').strip('\r').split(' ')
            box = [float(x) for x in line]
            # crop the image and save
            x1, x2 = int((box[2]-box[4]/2)*h), int((box[2]+box[4]/2)*h)
            y1, y2 = int((box[1]-box[3]/2)*w)+1, int((box[1]+box[3]/2)*w)+1
            cnt += 1
            cv2.imwrite('data/subimg/image/'+str(cnt)+'.png', src[x1:x2, y1:y2])
            sub_img = sub_img.append(
                {'id': cnt, 'class': label, 'FileName': s, 'GlobalID': GlobalID},
                ignore_index=True
            )
sub_img.to_csv('data/subimg/list.csv', index=False)
sub_img.head()

,id,class,FileName,GlobalID
0,1,0,ATT1000_A5A50BAB-A6EF-4576-A1F8-A07862AADE3A,{E0AE2F2A-38A5-463C-97B5-9F84A477F9AE}
1,2,0,ATT1000_A5A50BAB-A6EF-4576-A1F8-A07862AADE3A,{E0AE2F2A-38A5-463C-97B5-9F84A477F9AE}
2,3,0,ATT1001_8B9F8599-61EF-4690-B195-888BCD319F32,{1CD14B56-4046-44C4-B1B8-8FB07A9071E9}
3,4,0,ATT1002_7DFE65D0-B77A-4C7F-8849-685CEA02E862,{0C119F06-3578-4028-AA97-924CD22AAF51}
4,5,0,ATT1003_20200718_151437,{52700FB3-ADBF-4EDE-BE99-9DF57E40E73F}


In [9]:
import cv2
i = cv2.imread('data/image/ATT1_DSCN9647.png')
cv2.imshow("img", i)
cv2.waitKey(0)    
cv2.destroyAllWindows() 